In [187]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from pandas import DataFrame 
import os
cnt = 0
user_id,content_type,timestamp,textual=[],[],[],[]
with open('./Task2_user_reputation_prediction/Reputation_prediction_data_train.txt') as f:    
    for line in f:
        data = line.strip().split()
        user_id.append(data[-3])
        content_type.append(data[-2])
        timestamp.append(data[-1])
        textual.append(data[:-3])
df = DataFrame({'user_id': user_id})
# df['user_id']=user_id
# df['content_type'] = content_type
# df['timestamp'] = timestamp
df['textual'] = textual
df.shape

(95803, 2)

In [265]:
df = df.groupby('user_id').agg({'textual':'sum'})
agg_df=df['textual'].to_frame()
agg_df.index=agg_df.index.map(int)
agg_df.sort_index(inplace=True)

In [266]:
agg_df.textual

user_id
1       [main, tenet, belief, emerg, church, emerg, wo...
2       [christ, death, signific, question, titl, quit...
3       [long, jesus, tomb, christian, common, celebr,...
5       [pray, peopl, outsid, triniti, understand, rom...
6       [christian, bibl, literalist, line, bibl, lite...
7       [book, testament, chosen, interest, find, proc...
9       [good, news, refer, good, news, differ, jesus,...
10      [someon, heaven, faith, sin, forgiven, heaven,...
11      [absent, apostol, success, argument, author, c...
13      [origin, cathol, apocrypha, cathol, tradit, se...
14      [fast, daniel, cyrus, king, persia, reveal, da...
15      [differ, judg, god, question, god, answer, man...
17      [bibl, stipul, prioriti, person, allegi, fair,...
18      [origin, moral, civil, ceremoni, distinct, tes...
19      [physic, evid, global, flood, young, earth, cr...
21      [biblic, basi, total, deprav, calvin, point, s...
22      [role, music, corpor, worship, mani, music, ex...
23    

In [267]:
user_id, label = [],[]
with open('./Task2_user_reputation_prediction/Reputation_prediction_labels_train.txt') as f:    
    for line in f:
        data = line.strip().split()
        user_id.append(data[0])
        label.append(data[1])
df_test = DataFrame(index=user_id)
df_test['label']=label
df_test.head()


,label
1,1
2,1
3,1
5,1
6,1


In [268]:
agg_df['label']= df_test['label'].tolist()
agg_df['textual']=[' '.join(i) for i in agg_df.textual]

In [263]:
# import numpy as np
# from sklearn.cross_validation import train_test_split
# a_train, a_test, b_train, b_test = train_test_split(agg_df.textual,agg_df.label , test_size=0.2)
# vectorizer = CountVectorizer()
# X = vectorizer.fit_transform(a_train)
# from sklearn.naive_bayes import MultinomialNB
# clf = MultinomialNB().fit(X, b_train)

# Y = vectorizer.transform(a_test)

# predicted = clf.predict(Y)
# np.mean(predicted == b_test)  

In [269]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(agg_df.textual)
# X.toarray()

In [211]:
# from sklearn.feature_extraction.text import TfidfTransformer
# tf_transformer = TfidfTransformer(use_idf=False).fit(X)
# X_train_tf = tf_transformer.transform(X)
# X_train_tf

In [270]:
import numpy as np
from sklearn.cross_validation import train_test_split
a_train, a_test, b_train, b_test = train_test_split(X,agg_df.label , test_size=0.1)
# train_data,test_data,train_label,test_label
# 0.1，6

In [258]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(a_train, b_train)
predicted = clf.predict(a_test)
np.mean(predicted == b_test)   

0.38400000000000001

In [29]:
# docs_new = ['westminst welfar week', 'worthless']
# X_new_counts = vectorizer.transform(docs_new)
# X_new_tfidf = tf_transformer.transform(X_new_counts)

# predicted = clf.predict(X_new_tfidf)

# for doc, category in zip(docs_new, predicted):
#      print('%r => %s' % (doc, category))

In [271]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),])

In [277]:
import numpy as np
from sklearn.cross_validation import train_test_split
a_train, a_test, b_train, b_test = train_test_split(agg_df.textual,agg_df.label, test_size=0.1,random_state=10)
text_clf.fit(a_train, b_train)  

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [282]:
import numpy as np
predicted = text_clf.predict(a_test)
np.mean(predicted == b_test)   

0.30399999999999999

In [316]:
from sklearn.linear_model import SGDClassifier
from sklearn import svm
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
#                      ('svm', svm.SVC())
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, random_state=17,
                                           max_iter=5, tol=None)),
])
text_clf.fit(a_train, b_train)  

predicted = text_clf.predict(a_test)
np.mean(predicted == b_test)

0.52400000000000002

In [288]:
from sklearn import metrics
print(metrics.classification_report(a_test, predicted,
    ))

/Users/hh/anaconda/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/hh/anaconda/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [317]:
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
              'tfidf__use_idf': (True, False),
              'clf__alpha': (1e-2, 1e-3),
}

In [318]:
gs_clf = GridSearchCV(text_clf, parameters, cv=5, iid=False, n_jobs=-1)

In [319]:
gs_clf = gs_clf.fit(a_train, b_train)

In [320]:
# gs_clf.predict(['God is love'])[0]
gs_clf.best_score_                                  
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

clf__alpha: 0.001
tfidf__use_idf: True
vect__ngram_range: (1, 1)


In [321]:
gs_clf.best_score_    

0.56535319845200882